## 1- importer les bibliothéques

In [2]:
import os
from PIL import Image
import numpy as np 
import pandas as pd
from skimage.io import imread
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import make_scorer, recall_score, precision_score
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation, BatchNormalization
from keras.backend import dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.optimizers import Adam
from sklearn.metrics import classification_report


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 2- fixer les path des données augmenter 

In [3]:
sj_fliped='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_fliped'
sj_zoom='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_zoom'
sj_rotation='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_rotation'
sj_translation='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_translation'
sj_distortion='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_distortion'
sj_rot_trans_flip='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_fliped_rotation_translation'
sj_flip_distortion='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_fliped_distortion'
sj_rot_tran='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/sj_aug/sj_rotation_translation'

nsj_fliped='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_fliped'
nsj_zoom='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_zoom'
nsj_rotation='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_rotation'
nsj_translation='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_translation'
nsj_distortion='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_distortion'
nsj_rot_trans_flip='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_fliped_rotation_translation'
nsj_flip_distortion='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_fliped_distortion'
nsj_rot_tran='/kaggle/input/sjdata/DATA_(512,512)/augmented_data/nsj_aug/nsj_rotation_translation'

In [4]:
sj_path='/kaggle/input/sjdata/DATA_(512,512)/sj_data'
nsj_path='/kaggle/input/sjdata/DATA_(512,512)/nsj_data'

## Mise en Forme des images

In [5]:

def mise_tenseur (path, nl, nc,pl):
    images_liste = os.listdir(path)
    images_liste.sort()
    Nb_images = len(images_liste)
    tenseur = np.empty((Nb_images, nl, nc, pl), dtype=np.uint8)
    k = 0
    for im in images_liste:
       
        img = imread(path+'/'+im)
        tenseur[k, :, :, 0] = img
        k +=1
    return(tenseur)

In [6]:
sj_data = []
sj_data.extend(mise_tenseur(sj_rotation,512,512,1))
sj_data.extend(mise_tenseur(sj_fliped, 512,512,1))
sj_data.extend(mise_tenseur(sj_zoom,512,512,1))
sj_data.extend(mise_tenseur(sj_translation,512,512,1))
sj_data.extend(mise_tenseur(sj_distortion,512,512,1))
sj_data.extend(mise_tenseur(sj_rot_trans_flip,512,512,1))
sj_data.extend(mise_tenseur(sj_flip_distortion,512,512,1))
sj_data.extend(mise_tenseur(sj_path,512,512,1))

In [7]:
nsj_data = []
nsj_data.extend(mise_tenseur(nsj_zoom,512,512,1))
nsj_data.extend(mise_tenseur(nsj_fliped,512,512,1))
nsj_data.extend(mise_tenseur(nsj_translation,512,512,1))
nsj_data.extend(mise_tenseur(nsj_distortion,512,512,1))
nsj_data.extend(mise_tenseur(nsj_rot_trans_flip,512,512,1))
nsj_data.extend(mise_tenseur(nsj_flip_distortion,512,512,1))
nsj_data.extend(mise_tenseur(nsj_rotation,512,512,1))
nsj_data.extend(mise_tenseur(nsj_path,512,512,1))

In [8]:
len(nsj_data)

1760

In [9]:
len(sj_data)

1840

In [10]:
labels1 = [0] * len(nsj_data)  # labels pour la première classe (0)
labels2 = [1] * len(sj_data)  # labels pour la deuxième classe (1)

# Fusionner les données et les labels en un seul tableau multidimensionnel
data = np.concatenate([nsj_data, sj_data])
print(len(data))
labels = np.concatenate([labels1, labels2])
len(labels)


3600


3600

# cross validation

In [11]:
# définir la fonction pour créer le modèle
def create_model():
    model3 = Sequential()
    model3.add(Conv2D(32,(11,11),activation='relu', input_shape = (512,512,1)))
    model3.add(MaxPooling2D((2, 2)))
    model3.add(Conv2D(32, kernel_size = (7,7),activation='relu'))
    model3.add(MaxPooling2D((2, 2)))
    model3.add(Conv2D(32, kernel_size = (7,7),activation='relu'))
    model3.add(MaxPooling2D((2, 2)))
    model3.add(Conv2D(16, kernel_size = (7,7),activation='relu'))
    model3.add(MaxPooling2D((2, 2)))
    model3.add(Conv2D(16, kernel_size = (7,7),activation='relu'))
    model3.add(MaxPooling2D((2, 2)))
    model3.add(Flatten())
    model3.add(Dense(128,activation='relu'))
    model3.add(Dense(64,activation='relu'))
    model3.add(Dense(16, activation='relu'))
    model3.add(Dense(1,activation='sigmoid'))
    model3.compile(optimizer='adam',
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
    return model3


# Définition des listes pour stocker les mesures de performance pour chaque fold
precisions = []
recalls = []
f1_scores = []
lr_plus = []
lr_minus = []

# Définition de la cross-validation
kfold = KFold(n_splits=7, shuffle=True, random_state=42)

# Boucle sur les folds
for fold, (train, test) in enumerate(kfold.split(data, labels)):
    # Création du modèle
    model = create_model()
    # Entraînement du modèle sur les données d'entraînement du fold
    model.fit(data[train], labels[train], epochs=30, batch_size=32, verbose=0)
    # Affichage des résultats
    score = model.evaluate(data[test], labels[test], verbose=0)
    # Évaluation du modèle sur les données de test du fold
    y_pred = model.predict(data[test])
    y_pred_classes = np.where(y_pred>0.5,1.,0)
 # Calcul des mesures de performance
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels[test], y_pred_classes, average='binary')
    tn, fp, fn, tp = confusion_matrix(labels[test], y_pred_classes).ravel()
    lr_plus_fold = recall / (1 - tn / (tn + fp))
    lr_minus_fold = (1 - recall) / (tn / (tn + fp))
    # Stockage des mesures de performance pour le fold actuel
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1_score)
    lr_plus.append(lr_plus_fold)
    lr_minus.append(lr_minus_fold)
    # Affichage des résultats
    print('Fold', fold+1, '- Loss:', score[0], '- Accuracy:', score[1])
    print('Precision:', precision, '- Recall:', recall, '- F1 score:', f1_score)
    print('LR+:', lr_plus, '- LR-:', lr_minus)

17/17 [==============================] - 1s 38ms/step
Fold 1 - Loss: 0.13341079652309418 - Accuracy: 0.984466016292572
Precision: 0.9915254237288136 - Recall: 0.975 - F1 score: 0.9831932773109243
LR+: [134.06249999999915] - LR-: [0.025183150183150208]
17/17 [==============================] - 1s 39ms/step
Fold 2 - Loss: 0.04245053976774216 - Accuracy: 0.9883494973182678
Precision: 0.9886363636363636 - Recall: 0.9886363636363636 - F1 score: 0.9886363636363636
LR+: [134.06249999999915, 82.71590909090916] - LR-: [0.025183150183150208, 0.011501099706744858]
17/17 [==============================] - 1s 38ms/step
Fold 3 - Loss: 0.34215936064720154 - Accuracy: 0.8832684755325317
Precision: 0.8992537313432836 - Recall: 0.8795620437956204 - F1 score: 0.8892988929889298
LR+: [134.06249999999915, 82.71590909090916, 7.818329278183289] - LR-: [0.025183150183150208, 0.011501099706744858, 0.13570473938521643]
17/17 [==============================] - 1s 39ms/step
Fold 4 - Loss: 0.0205523744225502 - Accu

In [13]:
# Affichage des résultats
print('Precision : %.3f' % np.mean(precisions))
print('Recall : %.3f' % np.mean(recalls))
print('F1-score : %.3f' % np.mean(f1_scores))
print('LR+ : %.3f' % np.mean(lr_plus))
print('LR- : %.3f' % np.mean(lr_minus))

Precision : 0.956
Recall : 0.888
F1-score : 0.910
LR+ : 81.994
LR- : 0.119
